In [2]:
import cv2
import numpy as np
from PIL import Image

def convert_rgb_to_bgr(image):
    bgr_image = np.zeros_like(image)
    
    # Swap the channels to convert from RGB to BGR
    bgr_image[:,:,0] = image[:,:,2]
    bgr_image[:,:,1] = image[:,:,1]
    bgr_image[:,:,2] = image[:,:,0]
    
    return bgr_image

def mask_image(image, lower_bound, upper_bound):
    image_copy = image
    
    # Convert the image to HSV color space
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros_like(image_hsv, dtype=np.uint8)
    
    # Iterate through the image pixels
    for i in range(image_hsv.shape[0]):
        for j in range(image_hsv.shape[1]):
            # Check if the pixel values are within the specified range
            if (image_hsv[i,j,0] >= lower_bound[0] and image_hsv[i,j,1] >= lower_bound[1] and image_hsv[i,j,2] >= lower_bound[2]
                    and image_hsv[i,j,0] <= upper_bound[0] and image_hsv[i,j,1] <= upper_bound[1] and image_hsv[i,j,2] <= upper_bound[2]):
                # Set the corresponding values in the mask
                mask[i,j,0] = image_hsv[i,j,0]
                mask[i,j,1] = image_hsv[i,j,1]
                mask[i,j,2] = image_hsv[i,j,2]
    
    # Apply the mask to the original image
    masked_image = np.bitwise_and(mask, image_hsv)
    
    # Convert the masked image back to RGB color space
    masked_image = cv2.cvtColor(masked_image, cv2.COLOR_HSV2RGB)
    
    return masked_image

def convolution_3d(image, kernel):
    output = np.zeros_like(image)
    padded_image = np.zeros((image.shape[0]+kernel.shape[0]-1, image.shape[1]+kernel.shape[1]-1, image.shape[2]))
    padded_image[kernel.shape[0]-2:-1, kernel.shape[1]-2:-1, :] = image
    padded_image[0,0,:] = image[0,0,:]
    padded_image[-1,-1,:] = image[-1,-1,:]
    
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            for z in range(image.shape[2]):
                output[y, x, z] = (kernel * padded_image[y:y+kernel.shape[0], x:x+kernel.shape[1], z]).sum()
    
    return output


gaussian_blur_kernel = np.array([[1, 4, 6, 4, 1],
                                 [4, 16, 24, 16, 4],
                                 [6, 24, 36, 24, 6],
                                 [4, 16, 24, 16, 4],
                                 [1, 4, 6, 4, 1]]) / 256

# Set the file name for the input image
file_name = "mask.jpg"
im = np.array(Image.open(file_name))

# Set the lower and upper bounds for the blue color range in HSV
lower_blue = np.array([94, 130, 38])
upper_blue = np.array([179, 255, 255])

im = convert_rgb_to_bgr(im)
im = mask_image(convolution_3d(im, gaussian_blur_kernel), lower_blue, upper_blue)

# Convert the NumPy array to a PIL image
pil_img = Image.fromarray(im.astype(np.uint8))
pil_img.save('masked_image.jpeg')